<a href="https://colab.research.google.com/github/BuczynskiRafal/pipe_calculations/blob/main/Pipe_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rainwater_drainage_calculations-0.1.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./rainwater_drainage_calculations-0.1.tar.gz
  Created wheel for rainwater-drainage-calculations: filename=rainwater_drainage_calculations-0.1-py3-none-any.whl size=4694 sha256=8919e5983a12ce6c1b7c2d8cee0c5d6b8054f348384bbb35fbca3cdd37af863f
  Stored in directory: /root/.cache/pip/wheels/e2/43/f1/ced1796fe70b639e8a7490c7715bc2e7773cde56b8080b086f
Successfully built rainwater-drainage-calculations


# Imports





In [2]:
import math
import requests
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.lib.function_base import angle
from numpy import sin, cos, pi, linspace
from rainwater_drainage_calculations import calculations as pipe

logger = logging.getLogger(__name__)

# Insert data

In [3]:
raw_data = pd.read_excel('pipes_before_validation.xlsx')
data = raw_data.copy()
df = pd.DataFrame(data)
df.rename(columns=df.iloc[0]).drop(df.index[0])
df.head(10)
df['Flow [l/s]'] = df['Flow [l/s]'].astype(np.float64)


In [4]:
df.shape

(20, 10)

# Calculate pipe filling - h [m]

In [5]:
df.dtypes

Lp.                      int64
Start node              object
End node                object
Flow [l/s]             float64
Land elevation  [m]    float64
Channel bottom [m]     float64
Diameter [m]           float64
pipe dip [m]           float64
Length [m]             float64
slope [‰]              float64
dtype: object

In [6]:
df['h [m]'] = df.apply(lambda x: pipe.calc_h(q=x['Flow [l/s]'], d=x['Diameter [m]'], i=x['slope [‰]']), axis=1)
df.head(10)

,Lp.,Start node,End node,Flow [l/s],Land elevation [m],Channel bottom [m],Diameter [m],pipe dip [m],Length [m],slope [‰],h [m]
0,1,W1,S1,26.0,149.08,147.28,0.20,1.80,8.10,50.560000,0.0822
1,2,W2,S1,18.0,149.06,147.96,0.20,1.10,8.20,109.756098,0.0552
2,3,S1,S2,44.0,149.32,146.47,0.60,2.85,27.10,2.214022,0.1584
3,4,W3,S2,24.0,148.86,147.12,0.20,1.74,11.10,24.324324,0.0965
4,5,W4,S2,19.0,148.82,147.80,0.20,1.02,5.60,35.714286,0.0761
5,6,S2,S3,87.0,148.92,146.41,0.60,2.51,38.10,1.574803,0.2479
6,7,W5,S3,16.0,148.61,146.82,0.25,1.79,16.14,10.383387,0.0878
7,8,W6,S3,15.0,148.61,146.85,0.25,1.76,12.52,10.383387,0.0849
8,9,S3,S4,118.0,148.66,146.24,0.60,2.42,40.00,4.250000,0.2234
9,10,S4a,S4b,14.0,148.48,145.96,0.40,2.52,20.90,1.000000,0.1254


# Calculate max pipe filling - h max [m]

In [7]:
df['h max [m]'] = df.apply(lambda x: pipe.max_h(d=x['Diameter [m]']), axis=1)
df.head()

,Lp.,Start node,End node,Flow [l/s],Land elevation [m],Channel bottom [m],Diameter [m],pipe dip [m],Length [m],slope [‰],h [m],h max [m]
0,1,W1,S1,26.0,149.08,147.28,0.2,1.80,8.1,50.560000,0.0822,0.12
1,2,W2,S1,18.0,149.06,147.96,0.2,1.10,8.2,109.756098,0.0552,0.12
2,3,S1,S2,44.0,149.32,146.47,0.6,2.85,27.1,2.214022,0.1584,0.45
3,4,W3,S2,24.0,148.86,147.12,0.2,1.74,11.1,24.324324,0.0965,0.12
4,5,W4,S2,19.0,148.82,147.80,0.2,1.02,5.6,35.714286,0.0761,0.12


# Calculate min slope - slope min [‰]


In [8]:
df['slope min [‰]'] = df.apply(lambda x: pipe.min_slope(h=x['h [m]'], d=x['Diameter [m]']), axis=1)
df.head()

,Lp.,Start node,End node,Flow [l/s],Land elevation [m],Channel bottom [m],Diameter [m],pipe dip [m],Length [m],slope [‰],h [m],h max [m],slope min [‰]
0,1,W1,S1,26.0,149.08,147.28,0.2,1.80,8.1,50.560000,0.0822,0.12,5.719814
1,2,W2,S1,18.0,149.06,147.96,0.2,1.10,8.2,109.756098,0.0552,0.12,7.837630
2,3,S1,S2,44.0,149.32,146.47,0.6,2.85,27.1,2.214022,0.1584,0.45,2.712511
3,4,W3,S2,24.0,148.86,147.12,0.2,1.74,11.1,24.324324,0.0965,0.12,5.116532
4,5,W4,S2,19.0,148.82,147.80,0.2,1.02,5.6,35.714286,0.0761,0.12,6.057392


# Calculate max slope - slope max [‰]

In [17]:
df['slope max [‰]'] = df.apply(lambda x: pipe.max_slope(d=x['Diameter [m]']), axis=1)
df.head()

,Lp.,Start node,End node,Flow [l/s],Land elevation [m],Channel bottom [m],Diameter [m],pipe dip [m],Length [m],slope [‰],h [m],h max [m],slope min [‰],slope max [‰],v [m],dip is valid,slope is valid,v is valid
0,1,W1,S1,26.0,149.08,147.28,0.2,1.80,8.1,50.560000,0.0822,0.12,5.719814,230.0,2.146177,1,1,1
1,2,W2,S1,18.0,149.06,147.96,0.2,1.10,8.2,109.756098,0.0552,0.12,7.837630,230.0,2.563153,0,0,1
2,3,S1,S2,44.0,149.32,146.47,0.6,2.85,27.1,2.214022,0.1584,0.45,2.712511,52.5,0.738516,1,1,0
3,4,W3,S2,24.0,148.86,147.12,0.2,1.74,11.1,24.324324,0.0965,0.12,5.116532,230.0,1.603442,1,1,1
4,5,W4,S2,19.0,148.82,147.80,0.2,1.02,5.6,35.714286,0.0761,0.12,6.057392,230.0,1.736123,0,0,1


# Calculate velocity

In [10]:
df['v [m]'] = df.apply(lambda x: pipe.calc_velocity(h=x['h [m]'], d=x['Diameter [m]'], i=x['slope [‰]']), axis=1)
df.head(10)

,Lp.,Start node,End node,Flow [l/s],Land elevation [m],Channel bottom [m],Diameter [m],pipe dip [m],Length [m],slope [‰],h [m],h max [m],slope min [‰],slope max [‰],v [m]
0,1,W1,S1,26.0,149.08,147.28,0.20,1.80,8.10,50.560000,0.0822,0.12,5.719814,230.0,2.146177
1,2,W2,S1,18.0,149.06,147.96,0.20,1.10,8.20,109.756098,0.0552,0.12,7.837630,230.0,2.563153
2,3,S1,S2,44.0,149.32,146.47,0.60,2.85,27.10,2.214022,0.1584,0.45,2.712511,52.5,0.738516
3,4,W3,S2,24.0,148.86,147.12,0.20,1.74,11.10,24.324324,0.0965,0.12,5.116532,230.0,1.603442
4,5,W4,S2,19.0,148.82,147.80,0.20,1.02,5.60,35.714286,0.0761,0.12,6.057392,230.0,1.736123
5,6,S2,S3,87.0,148.92,146.41,0.60,2.51,38.10,1.574803,0.2479,0.45,1.899316,52.5,0.789887
6,7,W5,S3,16.0,148.61,146.82,0.25,1.79,16.14,10.383387,0.0878,0.15,5.154353,167.5,1.042486
7,8,W6,S3,15.0,148.61,146.85,0.25,1.76,12.52,10.383387,0.0849,0.15,5.292437,167.5,1.024273
8,9,S3,S4,118.0,148.66,146.24,0.60,2.42,40.00,4.250000,0.2234,0.45,2.052738,52.5,1.232127
9,10,S4a,S4b,14.0,148.48,145.96,0.40,2.52,20.90,1.000000,0.1254,0.28,3.526977,90.0,0.416628


#  Validate pipe dip

In [41]:
df['dip is valid'] = df.apply(lambda x: 1 if x['pipe dip [m]'] >=1.2 else 0, axis=1)
print(f"Row count with invalid pipe dip: {len(df[df['dip is valid'] == 0])}")
df[df['dip is valid'] == 0]


Row count with invalid pipe dip: 3


,Lp.,Flow [l/s],Diameter [m],pipe dip [m],slope [‰],h [m],h max [m],slope min [‰],slope max [‰],v [m],dip is valid,slope is valid,v is valid,h is valid
1,2,18.0,0.2,1.10,109.756098,0.0552,0.12,7.837630,230.0,2.563153,0,1,1,1
4,5,19.0,0.2,1.02,35.714286,0.0761,0.12,6.057392,230.0,1.736123,0,1,1,1
12,13,18.0,0.2,1.15,194.339623,0.0478,0.12,8.862464,230.0,3.142397,0,1,1,1


# Validate slope 



In [42]:
df['slope is valid'] = df.apply(lambda x: 1 if x['slope max [‰]'] >= x['slope [‰]'] >= x['slope min [‰]'] else 0, axis=1)
print(f"Row count with invalid pipe slope: {len(df[df['slope is valid'] == 0])}")
df[df['slope is valid'] == 0]

Row count with invalid pipe slope: 4


,Lp.,Flow [l/s],Diameter [m],pipe dip [m],slope [‰],h [m],h max [m],slope min [‰],slope max [‰],v [m],dip is valid,slope is valid,v is valid,h is valid
2,3,44.0,0.6,2.85,2.214022,0.1584,0.45,2.712511,52.5,0.738516,1,0,0,1
5,6,87.0,0.6,2.51,1.574803,0.2479,0.45,1.899316,52.5,0.789887,1,0,0,1
9,10,14.0,0.4,2.52,1.000000,0.1254,0.28,3.526977,90.0,0.416628,1,0,0,1
19,20,244.0,0.6,2.58,NaN,0.0001,0.45,3750.291695,52.5,NaN,1,0,0,1


# Validate velocity

In [43]:
df['v is valid'] = df.apply(lambda x: 1 if 5 >= x['v [m]'] >= 0.8 else 0, axis=1)
print(f"Row count with invalid pipe velocity: {len(df[df['v is valid'] == 0])}")
df[df['v is valid'] == 0]

Row count with invalid pipe velocity: 4


,Lp.,Flow [l/s],Diameter [m],pipe dip [m],slope [‰],h [m],h max [m],slope min [‰],slope max [‰],v [m],dip is valid,slope is valid,v is valid,h is valid
2,3,44.0,0.6,2.85,2.214022,0.1584,0.45,2.712511,52.5,0.738516,1,0,0,1
5,6,87.0,0.6,2.51,1.574803,0.2479,0.45,1.899316,52.5,0.789887,1,0,0,1
9,10,14.0,0.4,2.52,1.000000,0.1254,0.28,3.526977,90.0,0.416628,1,0,0,1
19,20,244.0,0.6,2.58,NaN,0.0001,0.45,3750.291695,52.5,NaN,1,0,0,1


# Validate pipe filling - h [m]

In [29]:
df['h is valid'] = df.apply(lambda x: 1 if x['h [m]'] <= x['h max [m]'] else 0, axis=1)
print(f"Row count with invalid pipe filling: {len(df[df['h is valid'] == 0])}")
df.head()

Row count with invalid pipe filling: 0


,Lp.,Start node,End node,Flow [l/s],Land elevation [m],Channel bottom [m],Diameter [m],pipe dip [m],Length [m],slope [‰],h [m],h max [m],slope min [‰],slope max [‰],v [m],dip is valid,slope is valid,v is valid,h is valid
0,1,W1,S1,26.0,149.08,147.28,0.2,1.80,8.1,50.560000,0.0822,0.12,5.719814,230.0,2.146177,1,1,1,1
1,2,W2,S1,18.0,149.06,147.96,0.2,1.10,8.2,109.756098,0.0552,0.12,7.837630,230.0,2.563153,0,1,1,1
2,3,S1,S2,44.0,149.32,146.47,0.6,2.85,27.1,2.214022,0.1584,0.45,2.712511,52.5,0.738516,1,0,0,1
3,4,W3,S2,24.0,148.86,147.12,0.2,1.74,11.1,24.324324,0.0965,0.12,5.116532,230.0,1.603442,1,1,1,1
4,5,W4,S2,19.0,148.82,147.80,0.2,1.02,5.6,35.714286,0.0761,0.12,6.057392,230.0,1.736123,0,1,1,1


# Prepare output 

In [38]:
df_output = df.copy()
df.columns
df.drop(columns=['Start node', 'End node', 'Land elevation  [m]', 'Channel bottom [m]', 'Length [m]'], axis=1, inplace=True)

df_output.head()

,Lp.,Flow [l/s],Diameter [m],pipe dip [m],slope [‰],h [m],h max [m],slope min [‰],slope max [‰],v [m],dip is valid,slope is valid,v is valid,h is valid
0,1,26.0,0.2,1.80,50.560000,0.0822,0.12,5.719814,230.0,2.146177,1,1,1,1
1,2,18.0,0.2,1.10,109.756098,0.0552,0.12,7.837630,230.0,2.563153,0,1,1,1
2,3,44.0,0.6,2.85,2.214022,0.1584,0.45,2.712511,52.5,0.738516,1,0,0,1
3,4,24.0,0.2,1.74,24.324324,0.0965,0.12,5.116532,230.0,1.603442,1,1,1,1
4,5,19.0,0.2,1.02,35.714286,0.0761,0.12,6.057392,230.0,1.736123,0,1,1,1


# Save to excel file

In [40]:
df_output.to_excel('pipes_after_validation.xlsx')